In [34]:
import math
import numpy as np
from scipy.optimize import fsolve

Verify integration

In [56]:
def compute_int(y, sigma=1):
    return np.sqrt(np.pi/2) * sigma * (math.erf((y+3)/(sigma*np.sqrt(2))) - math.erf((y-3)/(sigma*np.sqrt(2))))
print(compute_int(-2.8))
x = np.linspace(-3, 3, 10000)
print(0.0006 * np.sum(np.exp(-np.square(x+2.8)/2)[:9999]))

1.4519887577232509
1.452137612552812


$Z_i \sim Ber(\theta)$\
$\beta_i \sim U(-a,a)$\
$Y_i|Z_i, \beta_i \sim N(z_i \circ \beta_i, \sigma^2)$

In [20]:
''' 
compute normalization constant
'''
def compute_c(y, theta, a, sigma):
    c = 2*a * (1-theta) * np.exp(-y**2 / (2 * sigma**2)) \
    + theta * np.sqrt(np.pi/2) * sigma * \
    (math.erf((y+a)/(sigma*np.sqrt(2))) - math.erf((y-a)/(sigma*np.sqrt(2))))
    return c

In [25]:
''' 
compute the cumulative probablity of the marginal distribution of beta
'''
def cumulative_prob(t, y, norm_c, theta, a, sigma):
    assert t>=-a and t<=a
    prob = (1-theta) * np.exp(-y**2 / (2 * sigma**2)) * (t+a) / norm_c \
        + theta * np.sqrt(np.pi/2) * sigma * \
            (math.erf((y+a)/(sigma*np.sqrt(2))) - math.erf((y-t)/(sigma*np.sqrt(2)))) / norm_c
    return prob

In [28]:
y = 2
theta = 0.05
a = 3
sigma = 1
c = compute_c(y, theta, a, sigma)
print(c)

0.876858004982628


In [29]:
cumulative_prob(-3, y, c, theta, a, sigma)

0.0

In [30]:
cumulative_prob(3, y, c, theta, a, sigma)

1.0

In [33]:
cumulative_prob(-2.8, y, c, theta, a, sigma)

0.029324893165522226

In [42]:
fsolve(lambda x: cumulative_prob(x, y, c, theta, a, sigma) - 0.025, -2)

array([-2.82949636])

In [38]:
fsolve(lambda x: cumulative_prob(x, y, c, theta, a, sigma) - 0.975, 2)

array([2.86380276])

If $a$ is too small, then the posterior CI will always contain 0.

In [43]:
y = 5
theta = 0.05
a = 10
sigma = 1
c = compute_c(y, theta, a, sigma)
print(c)

0.12540218421537278


In [57]:
cumulative_prob(-10, y, c, theta, a, sigma)

0.0

In [54]:
cumulative_prob(10, y, c, theta, a, sigma)

1.0

In [58]:
fsolve(lambda x: cumulative_prob(x, y, c, theta, a, sigma) - 0.025, 2)

array([3.0339417])

In [63]:
fsolve(lambda x: cumulative_prob(x, y, c, theta, a, sigma) - 0.975, 8)

array([6.96118773])

$Z_i \sim Ber(\theta)$\
$\beta_i|Z_i=1 \sim U(-a,a)$\
$\beta_i|Z_i=0 \sim \delta (0)$\
$Y_i|\beta_i \sim N( \beta_i, \sigma^2)$

In [80]:
''' 
compute normalization constant
'''
def compute_c(y, theta, a, sigma):
    c = 2*a * (1-theta) * np.exp(-y**2 / (2 * sigma**2)) \
    + theta * np.sqrt(np.pi/2) * sigma * \
    (math.erf((y+a)/(sigma*np.sqrt(2))) - math.erf((y-a)/(sigma*np.sqrt(2))))
    return c / (2*a)

In [93]:
''' 
compute the cumulative probablity of the marginal distribution of beta
'''
def cumulative_prob(t, y, norm_c, theta, a, sigma):
    assert t>=-a and t<=a
    if t>=0:
        prob = (1-theta) * np.exp(-y**2 / (2 * sigma**2)) / norm_c \
            + theta * np.sqrt(np.pi/2) * sigma * \
                (math.erf((y+a)/(sigma*np.sqrt(2))) - math.erf((y-t)/(sigma*np.sqrt(2)))) / (2 * a * norm_c)
    elif t<0:
        prob = theta * np.sqrt(np.pi/2) * sigma * \
                (math.erf((y+a)/(sigma*np.sqrt(2))) - math.erf((y-t)/(sigma*np.sqrt(2)))) / (2 * a * norm_c)
    return prob

In [120]:
y = 2
theta = 0.05
a = 3
sigma = 1
c = compute_c(y, theta, a, sigma)
print(c)

0.146143000830438


In [121]:
print(cumulative_prob(-3, y, c, theta, a, sigma))
print(cumulative_prob(3, y, c, theta, a, sigma))

0.0
1.0000000000000002


In [124]:
print(cumulative_prob(-0.01, y, c, theta, a, sigma))
print(cumulative_prob(0, y, c, theta, a, sigma))

0.0031752871211294924
0.8829963121990655


In [135]:
y = 4
theta = 0.05
a = 10
sigma = 1
c = compute_c(y, theta, a, sigma)
print(c)

0.006585260176902366


In [136]:
print(cumulative_prob(-10, y, c, theta, a, sigma))
print(cumulative_prob(10, y, c, theta, a, sigma))

0.0
1.0


In [137]:
print(cumulative_prob(0, y, c, theta, a, sigma))

0.048424505337170534


In [138]:
print(cumulative_prob(7, y, c, theta, a, sigma))

0.998715430366437


In [64]:
import numpy.random as rng
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
import pandas as pd
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [67]:
class Generator_eye(object):
    def __init__(self, p, theta, beta_range) -> None:
        self.p = p 
        self.theta = theta 
        self.beta_range = beta_range 
        self.X = np.eye(p)
    
    def generate_samples(self, n):
        scale = self.beta_range[1] - self.beta_range[0]
        theta = np.ones((n, self.p)) * self.theta
        gamma = rng.binomial(1, theta)
        beta = np.zeros((n, self.p))
        beta[gamma == 1] = rng.rand(np.sum(gamma == 1)) * scale + self.beta_range[0]
        beta[gamma == 0] = 0. 
        Y = beta@self.X.T + rng.randn(n, self.N)
        return gamma, beta, Y 

In [68]:
generator_eye = Generator_eye(p=200, theta=0.05, beta_range=(-10,10))

In [69]:
rng.seed(0)
gamma_train, beta_train, Y_train = generator_eye.generate_samples(1000000)
gamma_val, beta_val, Y_val = generator_eye.generate_samples(10000)

In [70]:
mean = Y_train.mean(0)
std = Y_train.std(0)
Y_train = (Y_train - mean) / std 
Y_val = (Y_val - mean) / std 

In [71]:
rng.seed(1)
gamma_test, beta_test, Y_test = generator_eye.generate_samples(10000)
Y_test = (Y_test - mean) / std

In [72]:
class MLP(nn.Module):
    def __init__(self, N, p):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(N, 1024)
        self.fc2 = nn.Linear(1024, 2048)
        self.fc3 = nn.Linear(2048, 2048)
        self.fc4 = nn.Linear(2048, 1024)
        self.fc5 = nn.Linear(1024, p)
        self.relu = nn.ReLU()
        self.mseloss = nn.MSELoss()
        self.bceloss = nn.BCEWithLogitsLoss()

    def forward(self, input):
        u = self.relu(self.fc1(input))
        u = self.relu(self.fc2(u))
        u = self.relu(self.fc3(u))
        u = self.relu(self.fc4(u))
        output = self.fc5(u)
        return output

    def get_mseloss(self, data, targ):
        output = self.forward(data)
        loss = self.mseloss(output, targ)
        return loss 
    
    def get_bceloss(self, data, targ):
        output = self.forward(data)
        loss = self.bceloss(output, targ)
        return loss 

    def get_quanloss(self, data, targ, tau):
        output = self.forward(data)
        errs = targ - output 
        loss = torch.mean(torch.max((tau-1)*errs, tau*errs))
        return loss 

In [73]:
''' 
 loss_type: 'mse' for posterior mean, 'bce' for predicting whether beta is 0, 'quantile' for posterior quantile.
q: Only used when loss_type is 'quantile', q quantile.
'''
def train_epoch(model, optimizer, train_data, train_labels, batch_size, loss_type, q):
    model.train()
    n = train_data.shape[0]
    train_loss = 0.
    for i in range(math.ceil(n/batch_size)):
        data = torch.from_numpy(train_data[(i*batch_size):min((i+1)*batch_size, n-1)]).type(torch.float).to(device)
        targ = torch.from_numpy(train_labels[(i*batch_size):min((i+1)*batch_size, n-1)]).type(torch.float).to(device)
        if loss_type == 'mse':
            loss = model.get_mseloss(data, targ)
        elif loss_type == 'bce':
            loss = model.get_bceloss(data, targ)
        elif loss_type == 'quantile':
            loss = model.get_quanloss(data, targ, q)
        train_loss += loss.item() * data.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return train_loss/n 

def model_test(model, test_data, test_labels, loss_type='mse', q=0.5):
    model.eval()
    with torch.no_grad():
        data = torch.from_numpy(test_data).type(torch.float).to(device)
        targ = torch.from_numpy(test_labels).type(torch.float).to(device)
        if loss_type == 'mse':
            loss = model.get_mseloss(data, targ)
        elif loss_type == 'bce':
            loss = model.get_bceloss(data, targ)
        elif loss_type == 'quantile':
            loss = model.get_quanloss(data, targ, q)
    return loss.item()

def train_model(model, lr, batch_size, epochs, train_data, train_labels, loss_type='mse', q=0.5, val_data=None, val_labels=None, early_stop=5):
    assert loss_type in ['mse', 'bce', 'quantile']
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train_losses = []
    val_losses = []
    min_loss = 1e6
    es_count = 0
    for i in range(epochs):
        train_loss = train_epoch(model, optimizer, train_data, train_labels, batch_size, loss_type, q)
        print('Epoch: {}'.format(i+1))
        print('Train loss: {:.5f}'.format(train_loss))
        train_losses.append(train_loss)
        if isinstance(val_data, np.ndarray):
            val_loss = model_test(model, val_data, val_labels, loss_type, q)
            print('Val loss: {:.5f}'.format(val_loss))
            val_losses.append(val_loss)
            if val_loss <= min_loss:
                min_loss = val_loss
                es_count = 0
            if es_count >= early_stop:
                break
            es_count += 1
    return train_losses, val_losses

def predict(model, Y):
    model.eval()
    with torch.no_grad():
        data = torch.from_numpy(Y).type(torch.float).to(device)
        pred = model(data)
    return pred.detach().cpu().numpy()

In [74]:
def show_loss(train_losses, val_losses):
    plt.plot(range(len(train_losses)), train_losses)
    plt.plot(range(len(train_losses)), val_losses)
    plt.legend(['train loss', 'val loss'], loc="upper right")
    plt.show()

def predict_class(model, Y):
    model.eval()
    with torch.no_grad():
        data = torch.from_numpy(Y).type(torch.float).to(device)
        pred = torch.sigmoid(model(data))
    return pred.detach().cpu().numpy()

In [77]:
md = MLP(N=200, p=200).to(device)
train_losses, val_losses = train_model(md, 0.001, 256, 80, Y_train, beta_train, val_data=Y_val, val_labels=beta_val)

Epoch: 1
Train loss: 33.24183
Val loss: 33.22535
Epoch: 2
Train loss: 33.09696
Val loss: 33.09955
Epoch: 3
Train loss: 32.96765
Val loss: 33.00992
Epoch: 4
Train loss: 32.88032
Val loss: 32.95216
Epoch: 5
Train loss: 32.81909
Val loss: 32.90902
Epoch: 6
Train loss: 32.77043
Val loss: 32.86763
Epoch: 7
Train loss: 32.72477
Val loss: 32.82511
Epoch: 8
Train loss: 32.68126
Val loss: 32.78724
Epoch: 9
Train loss: 32.64445
Val loss: 32.76188
Epoch: 10
Train loss: 32.61947
Val loss: 32.74752
Epoch: 11
Train loss: 32.60234
Val loss: 32.74432
Epoch: 12
Train loss: 32.58886
Val loss: 32.74987
Epoch: 13
Train loss: 32.57552
Val loss: 32.76115
Epoch: 14
Train loss: 32.56219
Val loss: 32.76849
Epoch: 15
Train loss: 32.54930
Val loss: 32.77956
Epoch: 16
Train loss: 32.53586
Val loss: 32.79289


In [75]:
md_q025 = MLP(N=200, p=200).to(device)
train_losses, val_losses = train_model(md_q025, 0.001, 256, 100, Y_train, beta_train, loss_type='quantile', q=0.025, val_data=Y_val, val_labels=beta_val)

Epoch: 1
Train loss: 0.24972
Val loss: 0.24457
Epoch: 2
Train loss: 0.24430
Val loss: 0.24489
Epoch: 3
Train loss: 0.24420
Val loss: 0.24418
Epoch: 4
Train loss: 0.24409
Val loss: 0.24413
Epoch: 5
Train loss: 0.24403
Val loss: 0.24410
Epoch: 6
Train loss: 0.24400
Val loss: 0.24412
Epoch: 7
Train loss: 0.24397
Val loss: 0.24414
Epoch: 8
Train loss: 0.24395
Val loss: 0.24409
Epoch: 9
Train loss: 0.24394
Val loss: 0.24403
Epoch: 10
Train loss: 0.24393
Val loss: 0.24408
Epoch: 11
Train loss: 0.24392
Val loss: 0.24404
Epoch: 12
Train loss: 0.24391
Val loss: 0.24403
Epoch: 13
Train loss: 0.24390
Val loss: 0.24400
Epoch: 14
Train loss: 0.24389
Val loss: 0.24400
Epoch: 15
Train loss: 0.24389
Val loss: 0.24400
Epoch: 16
Train loss: 0.24388
Val loss: 0.24401
Epoch: 17
Train loss: 0.24388
Val loss: 0.24404
Epoch: 18
Train loss: 0.24387
Val loss: 0.24405
Epoch: 19
Train loss: 0.24387
Val loss: 0.24405


In [76]:
md_q975 = MLP(N=200, p=200).to(device)
train_losses, val_losses = train_model(md_q975, 0.001, 256, 100, Y_train, beta_train, loss_type='quantile', q=0.975, val_data=Y_val, val_labels=beta_val)

Epoch: 1
Train loss: 0.24983
Val loss: 0.24426
Epoch: 2
Train loss: 0.24429
Val loss: 0.24434
Epoch: 3
Train loss: 0.24419
Val loss: 0.24399


KeyboardInterrupt: 

In [9]:
def compute_p(beta, y, theta, sigma):
    return theta * np.exp(-(y-beta)**2 / (2 * sigma**2)) + (1-theta) * np.exp(-y**2 / (2 * sigma**2))

In [10]:
beta = np.linspace(-3, 3, 10000)

In [11]:
p = compute_p(beta, 2, 0.05, 1)

In [12]:
0.0006 * np.sum(p[:9999]) / compute_c(2, 0.05, 1)

0.9998896233895808

In [66]:
rng.rand(3,2)

array([[0.51279774, 0.62133786],
       [0.0377208 , 0.0794336 ],
       [0.48984867, 0.19415153]])